# MSV Municipios Norte del Cauca

In [1]:
!pip install pandas
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn
!pip install scipy
!pip install numpy


[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import uniform, loguniform

In [9]:
path = 'dataset/datasetFinalTesisValle_NCauca.csv'
data = pd.read_csv(path)


Si se desea correr para 

76 => Valle del Cauca 

19 => Cauca

ó para todos quitar la condicion




In [10]:
df = data.loc[data['cod_dpto'] == 19].dropna(subset=['clasificacion_letter'])
#df = data.dropna(subset=['clasificacion_letter'])

In [11]:
print(df.head(3))

label_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
df['clasificacion_letter_numeric'] = df['clasificacion_letter'].map(label_mapping)

    cod_dpto  cod_mpio  cod_clase  cod_barrio  cod_vereda  cod_corregimiento  \
0         19     19455          2           0         333                  0   
11        19     19780          1         212           0                  0   
95        19     19110          3           0           0                  2   

    cod_comuna  tot_viviendas  tot_hogares  tip_vivienda  ...  ind_estudia  \
0           99              0            1             3  ...            9   
11          99              0            1             1  ...            9   
95          99              0            1             1  ...            9   

    clasificacion_letter  sum_gastos  sum_ingresos  ide_ficha_origen_hogar  \
0                      A           0             0  19455915512800003130-1   
11                     A           0             0  19780009530500000012-1   
95                     B      105000             0  19110273542300000457-1   

    max_nivel_educativo_en_hogar  alguien_hogar_emple

# Normalizar Columnas no Categoricas

In [12]:
X = df[
    ['edad_calculada','cod_dpto', 'cod_mpio', 'cod_clase', 'cod_barrio', 'cod_vereda', 'cod_corregimiento', 'cod_comuna', 'tip_vivienda',
     'tip_mat_paredes', 'tip_mat_pisos', 'ind_tiene_energia', 'ind_tiene_gas', 'num_cuartos_exclusivos',
     'num_cuartos_unicos_dormir', 'tip_sanitario', 'tip_uso_sanitario', 'tip_origen_agua', 'tip_uso_agua_beber',
     'ind_tiene_cocina', 'tip_prepara_alimentos', 'tip_uso_cocina', 'tip_energia_cocina',
      'num_personas_hogar', 'num_habita_vivienda', 'ind_tiene_internet', 'ind_evento_avalancha','ind_evento_inundacion',
     'max_nivel_educativo_en_hogar', 'alguien_hogar_empleado_o_similar', 'alguien_hogar_cotiza_pension',
     'sum_ingresos_hogar', 'sum_gastos_hogar']]
y = df['clasificacion_letter_numeric']


In [13]:
import pandas as pd

scaler = StandardScaler()

columns_to_normalize = ['sum_ingresos_hogar', 'sum_gastos_hogar']

X[columns_to_normalize] = scaler.fit_transform(X[columns_to_normalize])


/var/folders/vh/c65wsrm94x92d8rfk33vvpwr0000gn/T/ipykernel_49772/438794017.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[columns_to_normalize] = scaler.fit_transform(X[columns_to_normalize])


## Creacion del Modelo Base con columnas seleccionadas

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = SVC(probability=True, random_state=42)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import uniform, loguniform


param_distributions = {
    'C': loguniform(1e-3, 1e3),
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto'] + list(loguniform(1e-4, 1e0).rvs(10)),
    'degree': [2, 3, 4, 5],  # para kernel 'poly'
    'class_weight': ['balanced', None]
}

random_search = RandomizedSearchCV(
    model, param_distributions, n_iter=100, cv=5,
    scoring='accuracy', n_jobs=-1, random_state=42
)
random_search.fit(X_train, y_train)

print("Mejores parámetros de búsqueda aleatoria:", random_search.best_params_)
print("Mejor puntuación de búsqueda aleatoria:", random_search.best_score_)


best_params = random_search.best_params_
param_grid = {
    'C': [best_params['C'] / 10, best_params['C'], best_params['C'] * 10],
    'kernel': [best_params['kernel']],
    'gamma': [best_params['gamma']] if best_params['gamma'] in ['scale', 'auto'] else [best_params['gamma'] / 10, best_params['gamma'], best_params['gamma'] * 10],
    'degree': [best_params['degree']] if best_params['kernel'] == 'poly' else [3],
    'class_weight': [best_params['class_weight']]
}

grid_search = GridSearchCV(
    model, param_grid, cv=5, scoring='accuracy', n_jobs=-1
)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
print("Mejores parámetros finales:", grid_search.best_params_)
print("Mejor puntuación final:", grid_search.best_score_)

# 7. Evaluación del modelo
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# 8. Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión')
plt.ylabel('Etiqueta Verdadera')
plt.xlabel('Etiqueta Predicha')
plt.show()

# 9. Curva ROC
y_test_bin = label_binarize(y_test, classes=np.unique(y))
n_classes = y_test_bin.shape[1]

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green', 'orange']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (AUC = {1:0.2f})'.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC para clasificación multiclase')
plt.legend(loc="lower right")
plt.show()

# 10. Validación Cruzada
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5)
print(f"Puntuaciones de validación cruzada: {cv_scores}")
print(f"Media de validación cruzada: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# 11. Curva de aprendizaje
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores = learning_curve(
    best_model, X_train, y_train, cv=5, n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10))

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.title("Curva de Aprendizaje")
plt.xlabel("Tamaño del conjunto de entrenamiento")
plt.ylabel("Score")
plt.grid()
plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1, color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
plt.legend(loc="best")
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, learning_curve, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.inspection import permutation_importance
from sklearn.calibration import calibration_curve
from sklearn.model_selection import validation_curve
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import uniform, loguniform



# Análisis de características importantes
result = permutation_importance(best_model, X_test, y_test, n_repeats=10, random_state=42)
importance = pd.DataFrame({'feature': X.columns, 'importance': result.importances_mean})
importance = importance.sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=importance.head(15))
plt.title('Top 15 Características Más Importantes')
plt.show()

# Curvas de calibración
prob_pos = best_model.predict_proba(X_test)[:, 1]
fraction_of_positives, mean_predicted_value = calibration_curve(y_test, prob_pos, n_bins=10)

plt.figure(figsize=(10, 6))
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(mean_predicted_value, fraction_of_positives, marker='.')
plt.xlabel('Probabilidad predicha')
plt.ylabel('Fracción de positivos')
plt.title('Curva de Calibración')
plt.show()

# Análisis de errores
y_pred = best_model.predict(X_test)
incorrect = X_test[y_pred != y_test]
print("Número de predicciones incorrectas:", len(incorrect))
print("Características de las predicciones incorrectas:")
print(incorrect.describe())

# Validación cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(best_model, X_train, y_train, cv=cv)
print(f"Puntuaciones de validación cruzada estratificada: {cv_scores}")
print(f"Media de validación cruzada estratificada: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Curvas de precisión-recall
precision = dict()
recall = dict()
average_precision = dict()

for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_test_bin[:, i], y_pred_proba[:, i])
    average_precision[i] = average_precision_score(y_test_bin[:, i], y_pred_proba[:, i])

plt.figure(figsize=(10, 8))
for i in range(n_classes):
    plt.plot(recall[i], precision[i],
             label=f'Precision-Recall curve of class {i} (AP = {average_precision[i]:0.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curvas de Precisión-Recall para múltiples clases')
plt.legend(loc="best")
plt.show()

# Análisis de sensibilidad de hiperparámetros
param_range = np.logspace(-3, 3, 7)
train_scores, test_scores = validation_curve(
    SVC(), X_train, y_train, param_name="C", param_range=param_range,
    cv=5, scoring="accuracy", n_jobs=-1)

plt.figure(figsize=(10, 6))
plt.title("Curva de Validación para SVM")
plt.xlabel("C")
plt.ylabel("Score")
plt.semilogx(param_range, np.mean(train_scores, axis=1), label="Training score")
plt.semilogx(param_range, np.mean(test_scores, axis=1), label="Cross-validation score")
plt.legend(loc="best")
plt.show()

#Tiempos de entrenamiento y predicción
import time

start_time = time.time()
best_model.fit(X_train, y_train)
train_time = time.time() - start_time

start_time = time.time()
best_model.predict(X_test)
predict_time = time.time() - start_time

print(f"Tiempo de entrenamiento: {train_time:.2f} segundos")
print(f"Tiempo de predicción: {predict_time:.2f} segundos")

#Pruebas de estabilidad
from sklearn.model_selection import RepeatedStratifiedKFold

rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
stability_scores = cross_val_score(best_model, X, y, cv=rskf)

print("Estadísticas de estabilidad:")
print(f"Media: {np.mean(stability_scores):.4f}")
print(f"Desviación estándar: {np.std(stability_scores):.4f}")
print(f"Mínimo: {np.min(stability_scores):.4f}")
print(f"Máximo: {np.max(stability_scores):.4f}")